# [Transfer Learning](https://en.wikipedia.org/wiki/Transfer_learning)

### What is it?
A ML framework for using already trained model from Task 1 to solve Ttask 2.

### When do I use it?
When I want to train Computer Vision / NLP model and I:
* do not have much data
* do not have much computing power
* do not have much expertise in the field

### Why should I NOT use it?
There could exist quite cheap 3rd party service for my application from AWS etc.

# How does it work?

# Neureal Nets 2

Y'all remember the old Multi Layer Perceptron with flatten pixel vector on input...

## Multi Layer Perceptron img
<img src="nn_example.png" alt="Drawing" style="width: 600px;"/>

Thats not really what is used right now...

# Convolutional Neural Nets

An Nural Network design which enables for spatial relations of pixels (edges, curvatures, ...) to figure in the model.

## Convolution

#### Math definition:
<img src="convolution.png" alt="Drawing" style="width: 300px;"/>

#### Meaning:
Integral is basicaly a sum in continuous world => **Convolution is basicaly averaging** function f by a kernel function g

#### In 2D image:
Convolution is just averaging the picture by a small frame.
<img src="convolution2.png" alt="Drawing" style="width: 300px;"/>

#### Example: Sobel kernel 
(-1,0,1) \
(-2,0,2) \
(-1,0,1)

Enhances edges:

<img src="applying-sobel-kernel.png" alt="Drawing" style="width: 700px;"/>

## Pooling

Is sliding kernel operation for size reduction (pretty much like convolution). Average or Max pooling is used.

## CNN design

<img src="cnn_schema.png" alt="Drawing" style="width: 900px;"/>

#### Google's Inception V3 model to ilustrate complexity

<img src="inception_v3_architecture.png" alt="Drawing" style="width: 900px;"/>

## Can we hack it? => Yes, Transfer learning!

# Transfer learning


Some companies share their fine tuned models trained on dataset like ImageNet with **thousands** of general classes (eg. snake, locomotive, hourglass, ...) which would take days to train.

#### The idea is to: 
* **Remove last layer of trained model.** 
* **Keep the complex part, generating magic features.**
* **Retrain the last part of the net only (eg. with plain shallow NN).**

<img src="transfer_learning.png" alt="Drawing" style="width: 600px;"/>

#### This model will be trained in fraction of time and will perform reasonably good even with small amount of training data.

## Example with Keras:

In [ ]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# Retrain ResNet model for 2 classes
# get resnet50 from https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
# add model with already trained weights
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
# add shallow which we will train
my_new_model.add(Dense(num_classes, activation='softmax'))

# Disable to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

In [ ]:
# Compile TF model
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
image_size = 224
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# add new training data - rural / urban photos
train_generator = data_generator.flow_from_directory(
        '../input/urban-and-rural-photos/rural_and_urban_photos/train',
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

# add validation data
validation_generator = data_generator.flow_from_directory(
        '../input/urban-and-rural-photos/rural_and_urban_photos/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

# fit the new model
my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        validation_data=validation_generator,
        validation_steps=1)

## Sources
[Convolution Nets](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53)

[Wiki - Transfer Learning](https://en.wikipedia.org/wiki/Transfer_learning)

[TF - retrain.py example](https://www.tensorflow.org/hub/tutorials/image_retraining)

[Transfer Learning blogpost](https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a)

[Inception V3 model](https://github.com/tensorflow/models/tree/master/research/inception)